#### Librerias

In [105]:
import nltk
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.stem import WordNetLemmatizer

from sklearn.manifold import TSNE                   
import numpy as np       

from string import punctuation
from gensim.models import Word2Vec

#### Vamos a usar el copus del project gutenberg
El Project Gutenberg es un esfuerzo voluntario de digitalización de mas de 70000 libros gratuitos (dominio publico).

In [106]:
nltk.download('gutenberg')
gutenberg.readme().replace('\n', ' ')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\franco\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


'Project Gutenberg Selections http://gutenberg.net/  This corpus contains etexts from from Project Gutenberg, by the following authors:  * Jane Austen (3) * William Blake (2) * Thornton W. Burgess * Sarah Cone Bryant * Lewis Carroll * G. K. Chesterton (3) * Maria Edgeworth * King James Bible * Herman Melville * John Milton * William Shakespeare (3) * Walt Whitman  The beginning of the body of each book could not be identified automatically, so the semi-generic header of each file has been removed, and included below. Some source files ended with a line "End of The Project Gutenberg Etext...", and this has been deleted.  Information about Project Gutenberg (one page)  We produce about two million dollars for each hour we work.  The fifty hours is one conservative estimate for how long it we take to get any etext selected, entered, proofread, edited, copyright searched and analyzed, the copyright letters written, etc.  This projected audience is one hundred million readers.  If our value

In [107]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [108]:
bible_kjv_sents = gutenberg.sents('bible-kjv.txt')
len(bible_kjv_sents)

30103

#### Prepocesamiento

In [109]:
# creamos un objeto lemmatizador
lemmatizer = WordNetLemmatizer()

# No hace falta tokenizar, ya viene tokenizado el corpus

# Lemmatization
nltk_lemmaList = []
for sent in bible_kjv_sents:
    lemmatized_sent = [lemmatizer.lemmatize(word) for word in sent]
    nltk_lemmaList.append(lemmatized_sent)

# quitamos los signos de puntuacion
nltk_punctuation = [[word for word in sent if word not in punctuation] for sent in nltk_lemmaList] 

# Filtramos stopwords
nltk_stop_words = set(stopwords.words("english"))
discard_punctuation_and_lowercased_sents = [[word for word in sent if word not in nltk_stop_words] for sent in nltk_punctuation] 

In [110]:
discard_punctuation_and_lowercased_sents[3], len(discard_punctuation_and_lowercased_sents), type(discard_punctuation_and_lowercased_sents)

(['1', '1', 'In', 'beginning', 'God', 'created', 'heaven', 'earth'],
 30103,
 list)

#### Aplicamos el Word2Vec

In [112]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [113]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,# dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [114]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(discard_punctuation_and_lowercased_sents)

In [115]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 30103


In [116]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 5194


#### 3. Entrenamos los embeddings:

In [117]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(discard_punctuation_and_lowercased_sents,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 3659396.5
Loss after epoch 1: 2763785.0
Loss after epoch 2: 2622472.5
Loss after epoch 3: 2464285.0
Loss after epoch 4: 2421884.0
Loss after epoch 5: 2388022.0
Loss after epoch 6: 2248577.0
Loss after epoch 7: 2188622.0
Loss after epoch 8: 2164584.0
Loss after epoch 9: 2140060.0
Loss after epoch 10: 2111640.0
Loss after epoch 11: 2094198.0
Loss after epoch 12: 2078950.0
Loss after epoch 13: 2061242.0
Loss after epoch 14: 1850946.0
Loss after epoch 15: 1819236.0
Loss after epoch 16: 1802684.0
Loss after epoch 17: 1791264.0
Loss after epoch 18: 1780148.0
Loss after epoch 19: 1766968.0
Loss after epoch 20: 1758096.0
Loss after epoch 21: 1749408.0
Loss after epoch 22: 1747464.0
Loss after epoch 23: 1739832.0
Loss after epoch 24: 1730104.0
Loss after epoch 25: 1729268.0
Loss after epoch 26: 1719912.0
Loss after epoch 27: 1719748.0
Loss after epoch 28: 1717360.0
Loss after epoch 29: 1708668.0
Loss after epoch 30: 1711320.0
Loss after epoch 31: 1707608.0
Loss after epoch 3

(38907983, 48070500)

#### 4- Ensayar:

In [118]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["god"], topn=10)

[('goddess', 0.3828221261501312),
 ('Chemosh', 0.3552262783050537),
 ('wizard', 0.35330474376678467),
 ('Baalim', 0.34227436780929565),
 ('Diana', 0.33017948269844055),
 ('invention', 0.32000359892845154),
 ('graven', 0.3120911121368408),
 ('Baal', 0.30943194031715393),
 ('Ashtaroth', 0.3084787130355835),
 ('molten', 0.30496612191200256)]

Si vemos lo que significa cada palabras, encontramos cierta relación:
- goddess --> dios (femenino).
- Chemosh --> El nombre de un dios.
- Baal --> dios de la lluvia.
- Ashtaroth --> angel.

In [119]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["prayer"], topn=10)

[('Ham', 0.11829181760549545),
 ('blasting', 0.09445411711931229),
 ('whereas', 0.09057194739580154),
 ('cutteth', 0.07329064607620239),
 ('spared', 0.06312417984008789),
 ('nativity', 0.05512280389666557),
 ('hewn', 0.053010016679763794),
 ('reigned', 0.052299514412879944),
 ('beareth', 0.04619838669896126),
 ('divided', 0.042498137801885605)]

Si vemos lo que significa cada palabras, encontramos que son de relación inversa y también de poca relación:
- Ham --> Hijo de Noe, fue maldecido por Noe, puede conciderarse como un no orador (orador = positivo, Ham = negativo).  
El resto de las palabras tienen muy poca coincidencia como es de esperarse.

In [120]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["sacrifice"], topn=10)

[('offering', 0.38425588607788086),
 ('burnt', 0.3665766417980194),
 ('offer', 0.3520657420158386),
 ('offereth', 0.33148008584976196),
 ('sacrificed', 0.31127694249153137),
 ('fatlings', 0.30801793932914734),
 ('burn', 0.3022218346595764),
 ('savoury', 0.29598313570022583),
 ('draught', 0.2881058156490326),
 ('oblation', 0.28479206562042236)]

Si vemos lo que significa cada palabras, encontramos cierta relación:
- offering, offer, offereth --> ofrecer, puede ser ofrecer un sacrifio.
- burnt --> se puede quemar al sacrificio.  
- sacrificed --> esta palabra esta muy relaciona. 

In [146]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["altar"], topn=10)

[('grate', 0.4143572151660919),
 ('laver', 0.34282809495925903),
 ('tip', 0.33003225922584534),
 ('shittim', 0.32814842462539673),
 ('superscription', 0.32796451449394226),
 ('socket', 0.327086865901947),
 ('sprinkle', 0.3238252103328705),
 ('badger', 0.32369059324264526),
 ('board', 0.3221045136451721),
 ('fleshhooks', 0.32057031989097595)]

- fleshhooks (ganchos de carne)--> me parece que podría estar en un altar de sacrificios.
- board --> podría haber tableros en un altar.  
Igualmente en estos resultados no encuentro mucha proximidad. 

In [122]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

In [123]:
# el método `get_vector` permite obtener los vectores:
vector_God = w2v_model.wv.get_vector("god")
print(len(vector_God), vector_God)

300 [-7.03520656e-01 -2.83637404e-01 -1.02018462e-02 -6.53428972e-01
 -3.92749161e-01 -5.54828048e-01 -1.24949053e-01  4.64045197e-01
 -2.93092191e-01  3.46867621e-01  8.74506980e-02  5.15753984e-01
 -1.69843256e-01 -1.04016769e+00 -5.16713271e-03 -1.63220391e-01
 -1.35753769e-02  5.13582170e-01 -7.09954125e-04 -3.89447153e-01
 -5.56652308e-01 -9.74683613e-02 -1.56665266e-01  9.47714373e-02
 -1.73069626e-01  1.73681155e-01 -4.15116549e-01  3.30539018e-01
  1.24054611e-01  2.64678627e-01  1.95869863e-01  3.50541800e-01
  8.79925117e-02  1.88432842e-01 -2.71227926e-01  2.15670660e-01
  1.48772880e-01 -2.84983844e-01  1.72888830e-01  6.33225381e-01
 -5.47193736e-02  3.89761515e-02 -2.63760418e-01 -6.53275967e-01
 -7.12100342e-02 -5.60885742e-02  6.63954318e-01  8.13975155e-01
  9.55329537e-01 -1.98604632e-03  1.50509626e-01  1.86542377e-01
 -3.40532362e-01 -2.15571076e-01 -3.60463187e-02  2.77052701e-01
 -5.21707572e-02  1.52407065e-01  2.47868791e-01  1.16639100e-01
 -1.44767776e-01 -4.4

In [124]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_God)

[('god', 1.0),
 ('goddess', 0.382822185754776),
 ('Chemosh', 0.3552262783050537),
 ('wizard', 0.3533047139644623),
 ('Baalim', 0.34227436780929565),
 ('Diana', 0.33017948269844055),
 ('invention', 0.32000353932380676),
 ('graven', 0.3120911121368408),
 ('Baal', 0.30943194031715393),
 ('Ashtaroth', 0.3084787130355835)]

#### 5 - Visualizar agrupación de vectores

In [147]:
def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [148]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show()
#fig.show(renderer="colab") # esto para plotly en colab

En el grafico podemos ver que las palabras David y altar estan muy cercas entre si, según la biblia David construyó un altar e hizo sacrificios allí. Puede ser por esto esa relación que se ve en el grafico.  

En el grafico también se puede ver una cercanía entre 'seven' y 'priest' (sacerdote), tal vés esto se deba a los siete sacramentos y que la mayoria solo puede ser administrado por un sacerdote.

'water' y 'sea' estan cercanos.

In [149]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show()
#fig.show(renderer="colab") # esto para plotly en colab

#### test de analogías

In [139]:
# Palabras a testear
test_words = ['prayer', 'god', 'sacrifice', 'altar']

w2v_model.wv.n_similarity(['prayer', 'god'], ['sacrifice', 'altar']) # Compute cosine similarity between two sets of keys.

0.3106563

Existe algo de similitud en: 'orador' es a 'dios' com 'sacrificio' es a 'altar'.

In [140]:
# vectores de las distintas palabras
vector_prayer = w2v_model.wv.get_vector(test_words[0])
vector_god = w2v_model.wv.get_vector(test_words[1])
vector_sacrifice = w2v_model.wv.get_vector(test_words[2])
vector_altar = w2v_model.wv.get_vector(test_words[3])

vector_calculated = vector_god - vector_prayer + vector_sacrifice

In [141]:
w2v_model.wv.similar_by_vector(vector_calculated, topn=10)

[('god', 0.6052222847938538),
 ('sacrifice', 0.5744054317474365),
 ('nativity', 0.24366821348667145),
 ('goddess', 0.24013923108577728),
 ('fatted', 0.23854634165763855),
 ('whoring', 0.233870729804039),
 ('sacrificed', 0.23130129277706146),
 ('Sepharvaim', 0.23075631260871887),
 ('idol', 0.23040224611759186),
 ('gotten', 0.2302839756011963)]

Lo ideal hubiese sido obtener 'altar'. Se obtuvieron palabras mas relacionadas con 'sacrifice.'

In [142]:
w2v_model.wv.most_similar(vector_altar)

[('altar', 1.0000001192092896),
 ('grate', 0.4143572151660919),
 ('laver', 0.34282809495925903),
 ('tip', 0.33003225922584534),
 ('shittim', 0.32814842462539673),
 ('superscription', 0.32796451449394226),
 ('socket', 0.32708683609962463),
 ('sprinkle', 0.3238252103328705),
 ('badger', 0.32369062304496765),
 ('board', 0.3221045136451721)]

No hay ni una palabra en común entre el vector_altar y el vector_calculated.

In [136]:
# similitud coseno entre ambos vectores:
np.dot(vector_altar, vector_calculated) / (np.linalg.norm(vector_altar) * np.linalg.norm(vector_calculated))

0.13209791

In [145]:
# Otras analogias que haremos
analogies = [
    ("light", "darkness", "truth", "falsehood"),
    ("peace", "tranquility", "war", "conflict"),
    ("sin", "repentance", "disobedience", "forgiveness"),
    ("worship", "praise", "prayer", "supplication"),
    ("blessing", "grace", "curse", "judgment"),
    ("covenant", "promise", "sacrifice", "offering")
]

def cosine_similarity(vec1, vec2):

    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# Evaluamos las analogías
for word1, word2, word3, expected in analogies: # recorremos las analogias

    if all(word in w2v_model.wv for word in [word1, word2, word3, expected]): # nos fijamos si las palabras existen

        # hacemos el testeo
        vector_calculated = w2v_model.wv[word1] - w2v_model.wv[word2] + w2v_model.wv[word3]

        # nos quedamos con el primer valor
        most_similar = w2v_model.wv.similar_by_vector(vector_calculated, topn=5)

        # calculamos la similitud entre el valor esperado y el calculado
        similarity = cosine_similarity(vector_calculated, w2v_model.wv[expected])
        
        print(f"'{word1}' es a '{word2}' como '{word3}' es a '{expected}':")
        print(f"Esperado: {expected}")
        print(f"Calculado:")
        print(most_similar)
        print("\n")

    else:
        print(f"Una o más palabras en la analogía '{word1} - {word2} + {word3} = {expected}' no están en el vocabulario del modelo.")


'light' es a 'darkness' como 'truth' es a 'falsehood':
Esperado: falsehood
Calculado:
[('truth', 0.6526095271110535), ('light', 0.4564918279647827), ('Elias', 0.24968954920768738), ('sincerity', 0.24338467419147491), ('murmured', 0.24073129892349243)]


Una o más palabras en la analogía 'peace - tranquility + war = conflict' no están en el vocabulario del modelo.
'sin' es a 'repentance' como 'disobedience' es a 'forgiveness':
Esperado: forgiveness
Calculado:
[('disobedience', 0.5027846693992615), ('sin', 0.4916175305843353), ('rewardeth', 0.26877421140670776), ('flank', 0.26695147156715393), ('furious', 0.2584933042526245)]


'worship' es a 'praise' como 'prayer' es a 'supplication':
Esperado: supplication
Calculado:
[('prayer', 0.648455023765564), ('worship', 0.5898996591567993), ('supplication', 0.2981862723827362), ('wickedness', 0.28426113724708557), ('trance', 0.25848641991615295)]


Una o más palabras en la analogía 'sabbath - rest + work = labor' no están en el vocabulario del m

Para el caso de 'supplication' se predijo 'supplication' entre los resultados.  
   
Igualmente, en todos los casos obtenemos resultados para la palabra que esta relacionada con la objetivo, no propiamente con la palabra objetivo. Esto es, por ejemplo:  
- En el primer caso, se obtienen mas resultados acertados a 'truth' que 'falsehood'.